In [1]:
pip install mpld3

Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.dates as mdates
import mpld3
from IPython.display import HTML
import logging
import warnings



warnings.filterwarnings('ignore')
logging.getLogger('matplotlib.font_manager').disabled = True
matplotlib.rcParams['font.sans-serif'] = ['Arial Unicode MS']  # Replace 'SimHei' with the name of a font that supports Chinese on your system if necessary.
matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['axes.unicode_minus'] = False  # Ensure the minus sign is displayed correctly.



def load_and_process_data(file_path):
    data = pd.read_excel(file_path, engine='openpyxl')
    data['广告流水'] = data['广告流水'] / 10000  # Convert to "ten thousand"
    data['搜索广告流水'] = data['搜索广告流水'] / 10000
    data['广告流水 动态同环比变化率'] = data['广告流水 动态同环比变化率'].apply(lambda x: f"{x * 100:.2f}%")
    data['搜索广告流水 动态同环比变化率'] = data['搜索广告流水 动态同环比变化率'].apply(lambda x: f"{x * 100:.2f}%")
    return data

def get_top_20(data):
    # Exclude entries with "暂无brandtag" from the top 20 calculations
    filtered_data = data[data['Brandtag'] != "暂无brandtag"]
    # Calculate the 搜索占比 as a new column before filtering for the top 20
    filtered_data['搜索占比'] = (filtered_data['搜索广告流水'] / filtered_data['广告流水']) * 100
    filtered_data['搜索占比'] = filtered_data['搜索占比'].apply(lambda x: f"{x:.2f}%")  # Format as percentage

    top_20_ads = filtered_data.nlargest(20, '广告流水')
    top_20_search = filtered_data.nlargest(20, '搜索广告流水')

    top_20_ads['广告流水'] = top_20_ads['广告流水'].apply(lambda x: f"{x:.2f}万")
    top_20_search['搜索广告流水'] = top_20_search['搜索广告流水'].apply(lambda x: f"{x:.2f}万")

    top_20_ads = top_20_ads[['Brandtag', '广告流水', '广告流水 动态同环比变化率', '搜索占比']].rename(columns={'广告流水 动态同环比变化率': '消耗日环比'})
    top_20_search = top_20_search[['Brandtag', '搜索广告流水', '搜索广告流水 动态同环比变化率']].rename(columns={'搜索广告流水 动态同环比变化率': '搜索日环比'})

    return top_20_ads, top_20_search


def calculate_totals(data):
    total_ads_consumption = data['广告流水'].sum()
    total_search_revenue = data['搜索广告流水'].sum()
    return total_ads_consumption, total_search_revenue

def create_total_df(total_ads, total_search):
    return pd.DataFrame({
        '类型': ['广告流水', '搜索广告流水'],
        '总金额 (万)': [f"{total_ads:.2f}", f"{total_search:.2f}"]
    })

def style_dataframe(data):
    def style_specific_cell(value):
        if isinstance(value, str) and '%' in value:
            num_value = float(value.strip('%'))
            color = 'red' if num_value > 0 else 'green'
            return f'color: {color};'
        return ''
    # Check if the column exists to prevent KeyError
    columns_to_style = ['消耗日环比'] if '消耗日环比' in data.columns else []
    if '搜索日环比' in data.columns:
        columns_to_style.append('搜索日环比')
    return data.style.applymap(style_specific_cell, subset=columns_to_style).hide_index().render()


def generate_past_7days_data(today, folder_path):
    date_list = [(today - timedelta(days=i)).strftime("%Y-%m-%d") for i in range(1, 8)]
    daily_consumption = {date: 0 for date in date_list}
    date_list_reverse = [(today - timedelta(days=i)).strftime("%m%d") for i in range(1, 8)]

    for file_name in os.listdir(folder_path):
        if file_name.endswith('.xlsx'):
            file_date_str = file_name.split('.')[0]
            if file_date_str in date_list_reverse:
                file_date = today - timedelta(days=date_list_reverse.index(file_date_str) + 1)
                try:
                    file_data = load_and_process_data(os.path.join(folder_path, file_name))
                    daily_consumption[file_date.strftime("%Y-%m-%d")] += file_data['广告流水'].sum()
                except Exception as e:
                    print(f"Error reading file {file_name}: {e}")
    return pd.DataFrame(list(daily_consumption.items()), columns=['日期', '广告流水总数 (万)']).sort_values('日期')

def calculate_progress_from_july_first(today, folder_path):
    start_date = datetime(today.year, 7, 1)  # Set start date as July 1st of the current year
    total_consumption = 0
    
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.xlsx'):
            try:
                file_date_str = file_name.split('.')[0]
                # Append the current year to the file date string to create a full date
                file_date_str_with_year = file_date_str + str(today.year)
                file_date = datetime.strptime(file_date_str_with_year, "%m%d%Y")
                
                if file_date >= start_date:
                    file_path = os.path.join(folder_path, file_name)
                    file_data = load_and_process_data(file_path)
                    total_consumption += file_data['广告流水'].sum()  # Directly add without conversion
            except Exception as e:
                print(f"Error reading file {file_name}: {e}")
    return total_consumption  # Return the sum as is


def calculate_completion(today, folder_path, total_task_consumption=7710):
    current_progress = calculate_progress_from_july_first(today, folder_path)
    completion_rate = (current_progress / total_task_consumption) * 100  # Calculate completion rate as a percentage
    return current_progress, completion_rate

def plot_consumption_data(consumption_df):
    fig, ax = plt.subplots(figsize=(5, 3))  # Adjusted for better visibility
    consumption_df['日期'] = pd.to_datetime(consumption_df['日期'])
    ax.plot(consumption_df['日期'], consumption_df['广告流水总数 (万)'], marker='o', linestyle='-', color='blue', markersize=6)  # White line and markers

    for i, txt in enumerate(consumption_df['广告流水总数 (万)']):
        ax.annotate(f'{txt:.2f}', (consumption_df['日期'].iloc[i], consumption_df['广告流水总数 (万)'].iloc[i]),
                    textcoords="offset points", xytext=(0,10), ha='center', color='black')  # White annotations

    ax.set_facecolor('#121212')  # Dark background for the axes
    fig.patch.set_facecolor('#121212')  # Dark background for the figure
    ax.set_xlabel('日期', color='white')  # White x-axis label
    ax.set_ylabel('广告流水总数 (万)', color='black')  # White y-axis label
    ax.set_title('过去7日广告流水总数', color='black')  # White title
    ax.grid(True, linestyle='--', linewidth=0.5, color='grey')  # Grey grid lines for subtlety
    ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.xticks(rotation=45, color='black')  # White x-axis ticks
    plt.yticks(color='black')  # White y-axis ticks

    plt.savefig('daily_consumption_plot.png', transparent=True, bbox_inches='tight', pad_inches=0.1)  # Save with transparency
    plt.close(fig)
    return 'daily_consumption_plot.png'

def generate_html_output(today, top_20_ads_html, top_20_search_html, total_html, image_path, current_progress, completion_rate):
    css = """
    <style>
    body {
        background-color: #121212;
        color: #e0e0e0;
    }
    .table-container {
        display: flex;
        justify-content: space-between;
        align-items: flex-start;
        flex-wrap: nowrap;
        width: 100%;
    }
    .data-table, .chart-container {
        margin: 10px;
        background-color: transparent;
        border: 0px solid #555;
        padding: 10px;
        box-sizing: border-box;
        flex-basis: 31%;
    }
    .chart-container {
        align-items: flex-start;
    }
    .data-table th, .data-table td, .completion-section th, .completion-section td {
        padding: 8px;
        border: 1px solid #555;
        text-align: left;
    }
    .data-table th, .completion-section th {
        background-color: #333;
        font-weight: bold;
    }
    .heading {
        text-align: center;
        font-size: 24px;
        margin: 20px 0;
        position: relative;
    }
    .subheading {
        text-align: center;
        font-size: 18px;
        margin: 10px 0;
    }
    .date {
        position: absolute;
        top: 0;
        right: 0;
        font-size: 16px;
        color: #000000;
        margin: 20px;
    }
    .data-range {
        position: absolute;
        top: 0;
        left: 0;
        font-size: 16px;
        color: #121212;
        margin: 20px;
    }
    .completion-section {
        background-color: #ffffff; /* White background for the section */
        color: #000000; /* Black text for readability */
        border: 2px solid #555; /* Subtle border */
        padding: 10px;
        margin: 10px 0px;
        box-shadow: 0 4px 8px rgba(0,0,0,0.1); /* Adding some shadow for depth */
        border-radius: 8px; /* Rounded corners for aesthetics */
        text-align: center; /* Centering text horizontally */
        display: flex;
        flex-direction: column;
        justify-content: center; /* Centering text vertically */
        height: 100px; /* Adjust height as needed */
    }
    </style>
    """

    end_date = (today - timedelta(days=1)).strftime("%Y-%m-%d")
    start_date = (today - timedelta(days=7)).strftime("%Y-%m-%d")

    date_range_html = f'<div class="data-range">数据日期: {start_date} 至 {end_date}</div>'

    image_html = f'<img src="{image_path}" alt="Daily Consumption Plot" style="width:100%;">'
    
    completion_html = f'''
    <div class="completion-section">
        <div class="subheading">完成进度</div>
        总任务消耗: 7710万<br>
        现进度: {current_progress:.2f}万<br>
        完成率: {completion_rate:.2f}%
    </div>
    '''

    html_output = css + '<div class="heading">家居每日简报' + date_range_html + '<div class="date">' + today.strftime("%Y-%m-%d") + '</div></div>'
    html_output += '<div class="table-container">'
    html_output += '<div><div class="subheading">消耗排名 Top 20</div>' + top_20_ads_html + '</div>'
    html_output += '<div><div class="subheading">搜索排名 Top 20</div>' + top_20_search_html + '</div>'
    html_output += '<div class="chart-container"><div class="subheading">总消耗 & 图表</div>' + total_html + image_html+ '</div>'
    html_output += '</div>'
    return html_output



def main():
    today = datetime.today()
    folder_path = '/Users/zhuangmin/Desktop/日报原始数据'
    file_path = '/Users/zhuangmin/Desktop/日报原始数据/0730.xlsx'

    data = load_and_process_data(file_path)
    top_20_ads, top_20_search = get_top_20(data)
    total_ads, total_search = calculate_totals(data)
    total_df = create_total_df(total_ads, total_search)

    top_20_ads_html = style_dataframe(top_20_ads)
    top_20_search_html = style_dataframe(top_20_search)
    total_html = total_df.style.hide_index().render()

    past_7days_df = generate_past_7days_data(today, folder_path)
    line_chart_path = plot_consumption_data(past_7days_df)
    current_progress, completion_rate = calculate_completion(today, folder_path)

    html_output = generate_html_output(today, top_20_ads_html, top_20_search_html, total_html, line_chart_path, current_progress, completion_rate)
    display(HTML(html_output))

if __name__ == '__main__':
    main()

Brandtag,广告流水,消耗日环比,搜索占比
住范儿集团,9.16万,-12.50%,55.59%
一起装修网集团,7.79万,29.94%,15.01%
天坛集团,4.99万,76.83%,25.19%
尚层装饰集团,4.22万,-5.24%,21.26%
装小蜜集团,4.11万,-8.19%,44.35%
沪尚茗居集团,3.55万,-7.25%,51.22%
齐家网,3.53万,-6.83%,83.33%
圣都整装,3.29万,-14.40%,53.34%
梵客集团,3.25万,33.10%,17.00%
土巴兔,2.98万,29.35%,54.25%
